
# Monitoring Queries with the Database Console

Splice Machine jobs can be monitored, interrogated, and terminated in our Database Console. This notebook provides a quick introduction to the console, in these sections:

<ul class="italic">
    <li>Accessing the Console</li>
    <li>Drilling Down</li>
    <li>Terminating an Active Job</li>
</ul>



In [ ]:
%%sql 
-- QUERY 02
select
	s_acctbal,
	s_name,
	n_name,
	p_partkey,
	p_mfgr,
	s_address,
	s_phone,
	s_comment
from
	TPCH1.part,
	TPCH1.supplier,
	TPCH1.partsupp,
	TPCH1.nation,
	TPCH1.region
where
	p_partkey = ps_partkey
	and s_suppkey = ps_suppkey
	and p_size = 15
	and p_type like '%BRASS'
	and s_nationkey = n_nationkey
	and n_regionkey = r_regionkey
	and r_name = 'EUROPE'
	and ps_supplycost = (
		select
			min(ps_supplycost)
		from
			TPCH1.partsupp,
			TPCH1.supplier,
			TPCH1.nation,
			TPCH1.region
		where
			p_partkey = ps_partkey
			and s_suppkey = ps_suppkey
			and s_nationkey = n_nationkey
			and n_regionkey = r_regionkey
			and r_name = 'EUROPE'
	)
order by
	s_acctbal desc,
	n_name,
	s_name,
	p_partkey
{limit 100}
-- END OF QUERY


## Accessing the Database Console
You can access the Splice Machine Database Console by clicking the <span class="ConsoleLink">DB Console</span> button in your dashboard. The console initially displays the *Jobs* view, which looks like this:

<img class="fitwidth" src="https://doc.splicemachine.com/zeppelin/images/DBConsoleJobs1.png">

Some notes about the console display:

* Queries are reported as *Jobs*.
* Jobs that are still running are shown as *Active*, others are shown as *Completed*.
* Each Job has some number of *Stages*. You can see the stages in a job by clicking a link in the job description, or you can view all stages for all jobs by clicking the *Stages* tab at the top of the console screen.
* Each Stage has some number of *Tasks*, which are the most basic work units in the Spark Engine.

## Drilling Down

You can drill down by clicking links; for example, if you click on the topmost <span class="ConsoleLink">Produce Result Set</span> link in the above display, you'll see the *Job Details* screen, which displays the stages in the job:
<img class="fitwidth" src="https://doc.splicemachine.com/zeppelin/images/DBConsoleStages1.png">

### Viewing Stage Detail
You can click the topmost link in the Job Stages screen to display the *Stage Details* screen:
<img class="fitwidth" src="https://doc.splicemachine.com/zeppelin/images/DBConsoleStageDetail1.png">

### Viewing Stage Detail as a Directed Acyclic Graph (DAG)
You can switch to a graphical (DAG) depiction of the information in a screen; for example, here's a graph of one stage of collecting statistics:
<img class="fitwidth" src="https://doc.splicemachine.com/zeppelin/images/DBConsoleStageDag1.png">

### Viewing Job Stages Graphically
And here's a somewhat more interesting DAG view of a two-stage job:
<img class="fitwidth" src="https://doc.splicemachine.com/zeppelin/images/zepConsoleJobsDAG.png">

### Displaying a Timeline View
You can also view a timeline of the stage; for example:
<img class="fitwidth" src="https://doc.splicemachine.com/zeppelin/images/DBConsoleStageTimeline1.png">


## Terminating an Active Job

You can terminate an active job if you determine that something isn't working as expected. Simply access the *Jobs* tab while a job is actively running, and click the the *(kill)* text displayed next to the job description, as shown here:
<img class="fitwidth" src="https://doc.splicemachine.com/zeppelin/images/DBConsoleKill1.png">

You'll be asked to confirm that you want to terminate that job.



## Where to Go Next

The next notebook in this presentation introduces you to <a href="./3.4%20Transactions%20with%20Spark%20%26%20JDBC.ipynb">the transactional nature of Splice Machine and using JDBC to program transactions.</a>
